# Einstein's equations of general relativity in the [BSSN](http://www2.yukawa.kyoto-u.ac.jp/~yuichiro.sekiguchi/3+1.pdf) formalism, in ***curvilinear*** coordinates, using a covariant reference metric approach: C code generation of the evolution equations' right-hand sides

### ***Citations***: Generic curvilinear coordinate reference metric approach matches that of [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658), which is an extension of the spherical coordinate reference metric approach of [Baumgarte, Montero, Cordero-Carrión, and Müller (2012)](https://arxiv.org/abs/1211.6632), which builds upon the covariant "Lagrangian" BSSN formalism of [Brown (2009)](https://arxiv.org/abs/0902.3652). *See also citations within each article.*

**Background**: NRPy+'s original purpose was to be an easy-to-use code capable of generating Einstein's equations in a broad class of [singular](https://en.wikipedia.org/wiki/Coordinate_singularity), curvilinear coordinate systems, where the user need only input the scale factors of the underlying reference metric. Upon generating these equations, NRPy+ would then leverage SymPy's [common-expression-elimination (CSE)](https://en.wikipedia.org/wiki/Common_subexpression_elimination) and C code generation routines, coupled to its own [single-instruction, multiple-data (SIMD)](https://en.wikipedia.org/wiki/SIMD) functions, to generate highly-optimized C code.

This tutorial module demonstrates how Einstein's equations of general relativity in this formulation are constructed and output within NRPy+. As Einstein's equations in this formalism take the form of highly nonlinear, coupled *wave equations*, the [tutorial module on the scalar wave equation in curvilinear coordinates](Tutorial-ScalarWaveCurvilinear.ipynb) is *required* reading before beginning this module. That module, as well as its own prerequisite [module on reference metrics within NRPy+](Tutorial-Reference_Metric.ipynb) provides the needed overview of how NRPy+ handles reference metrics.

In summary, the equations we wish to input into NRPy+ are as follows (Eqs. 11 and 12 in [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658)):

\begin{align}
  \partial_{\perp} \bar{\gamma}_{i j} {} = {} & \frac{2}{3} \bar{\gamma}_{i j} \left (\alpha \bar{A}_{k}^{k} - \bar{D}_{k} \beta^{k}\right ) - 2 \alpha \bar{A}_{i j} \; , \\
  \partial_{\perp} \bar{A}_{i j} {} = {} & -\frac{2}{3} \bar{A}_{i j} \bar{D}_{k} \beta^{k} - 2 \alpha \bar{A}_{i k} {\bar{A}^{k}}_{j} + \alpha \bar{A}_{i j} K \nonumber \\
  & + e^{-4 \phi} \left \{-2 \alpha \bar{D}_{i} \bar{D}_{j} \phi + 4 \alpha \bar{D}_{i} \phi \bar{D}_{j} \phi \right . \nonumber \\
    & \left . + 4 \bar{D}_{(i} \alpha \bar{D}_{j)} \phi - \bar{D}_{i} \bar{D}_{j} \alpha + \alpha \bar{R}_{i j} \right \}^{\text{TF}} \; , \\
  \partial_{\perp} \phi {} = {} & \frac{1}{6} \left (\bar{D}_{k} \beta^{k} - \alpha K \right ) \; , \\
  \partial_{\perp} K {} = {} & \frac{1}{3} \alpha K^{2} + \alpha \bar{A}_{i j} \bar{A}^{i j} \nonumber \\
  & - e^{-4 \phi} \left (\bar{D}_{i} \bar{D}^{i} \alpha + 2 \bar{D}^{i} \alpha \bar{D}_{i} \phi \right ) \; , \\
  \partial_{\perp} \bar{\Lambda}^{i} {} = {} & \bar{\gamma}^{j k} \hat{D}_{j} \hat{D}_{k} \beta^{i} + \frac{2}{3} \Delta^{i} \bar{D}_{j} \beta^{j} + \frac{1}{3} \bar{D}^{i} \bar{D}_{j} \beta^{j} \nonumber \\
  & - 2 \bar{A}^{i j} \left (\partial_{j} \alpha - 6 \partial_{j} \phi \right ) + 2 \bar{A}^{j k} \Delta_{j k}^{i} \nonumber \\
  & -\frac{4}{3} \alpha \bar{\gamma}^{i j} \partial_{j} K \\
\partial_{0} \alpha &= -2 \alpha K \\
  \partial_{0} \beta^{i} &= B^{i} \\
  \partial_{0} B^{i} &= \frac{3}{4} \partial_{0} \bar{\Lambda}^{i} - \eta B^{i} \; .
\end{align}
where 
* the $\text{TF}$ superscript denotes the trace-free part.
* $\bar{\gamma}_{ij} = \varepsilon_{i j} + \hat{\gamma}_{ij}$, where $\bar{\gamma}_{ij} = e^{4\phi} \gamma_{ij}$ is the conformal metric, $\gamma_{ij}$ is the physical metric (see below), and $\varepsilon_{i j}$ encodes information about the non-hatted metric.
* $\gamma_{ij}$, $\beta^i$, and $\alpha$ are the physical (as opposed to conformal) spatial 3-metric, shift vector, and lapse, respectively, which may be defined via the 3+1 decomposition line element (in [$G=c=1$ units](https://en.wikipedia.org/wiki/Planck_units)):
$$ds^2 = -\alpha^2 dt^2 + \gamma_{ij}\left(dx^i + \beta^i dt\right)\left(dx^j + \beta^j dt\right).$$
* $\bar{R}_{ij}$ is the conformal Ricci tensor, computed via
\begin{align}
  \bar{R}_{i j} {} = {} & - \frac{1}{2} \bar{\gamma}^{k l} \hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j} + \bar{\gamma}_{k(i} \hat{D}_{j)} \bar{\Lambda}^{k} + \Delta^{k} \Delta_{(i j) k} \nonumber \\
  & + \bar{\gamma}^{k l} \left (2 \Delta_{k(i}^{m} \Delta_{j) m l} + \Delta_{i k}^{m} \Delta_{m j l} \right ) \; .
\end{align}
* $\partial_{\perp} = \partial_t - \mathcal{L}_\beta$; $\mathcal{L}_\beta$ is the [Lie derivative](https://en.wikipedia.org/wiki/Lie_derivative) along the shift vector $\beta^i$.
* $\partial_0 = \partial_t - \beta^i \partial_i$ is an advective time derivative.
* $\hat{D}_j$ is the [covariant derivative](https://en.wikipedia.org/wiki/Covariant_derivative) with respect to the reference metric $\hat{\gamma}_{ij}$.
* $\bar{D}_j$ is the [covariant derivative](https://en.wikipedia.org/wiki/Covariant_derivative) with respect to the barred spatial 3-metric $\bar{\gamma}_{ij}$
* $\Delta^i_{jk}$ is the tensor constructed from the difference of barred and hatted Christoffel symbols:
$$\Delta^i_{jk} = \bar{\Gamma}^i_{jk} - \hat{\Gamma}^i_{jk}$$
    * The related quantity $\Delta^i$ is defined $\Delta^i \equiv \bar{\gamma}^{jk} \Delta^i_{jk}$.
* $\bar{A}_{ij}$ is the conformal, trace-free extrinsic curvature: 
$$\bar{A}_{ij} = e^{-4\phi} \left(K_{ij} - \frac{1}{3}\gamma_{ij} K\right),$$
where $K$ is the trace of the extrinsic curvature $K_{ij}$.

## Numerical Implementation: Rescaling tensors to factor out coordinate singularities

While the above evolution equations are properly covariant (with the exception of the shift condition, which is a [freely specifiable gauge quantity](https://en.wikipedia.org/wiki/Gauge_fixing)), components of the rank-1 and rank-2 tensors $\varepsilon_{i j}$, $\bar{A}_{i j}$, and $\bar{\Lambda}^{i}$ will drop to zero (destroying information) or diverge (to $\infty$) at coordinate singularities. However, this behavior is well-understood in terms of the scale factors of the reference metric, enabling us to define rescaled version of these quantities that are well behaved (so that, e.g., they can be finite differenced).

For example, given a smooth vector *in a 3D Cartesian basis* $\bar{\Lambda}^{i}$, all components $\bar{\Lambda}^{x}$, $\bar{\Lambda}^{y}$, and $\bar{\Lambda}^{z}$ will be smooth (by assumption). When changing the basis to spherical coordinates (applying the appropriate Jacobian matrix transformation), we will find that since $\phi = \arctan(y/x)$, $\bar{\Lambda}^{\phi}$ is given by

\begin{align}
\bar{\Lambda}^{\phi} &= \frac{\partial \phi}{\partial x} \bar{\Lambda}^{x} + 
\frac{\partial \phi}{\partial y} \bar{\Lambda}^{y} + 
\frac{\partial \phi}{\partial z} \bar{\Lambda}^{z} \\
&= -\frac{y}{\sqrt{x^2+y^2}} \bar{\Lambda}^{x} + 
\frac{x}{\sqrt{x^2+y^2}} \bar{\Lambda}^{y} \\
&= -\frac{y}{r \sin\theta} \bar{\Lambda}^{x} + 
\frac{x}{r \sin\theta} \bar{\Lambda}^{y}.
\end{align}

Thus $\bar{\Lambda}^{\phi}$ diverges at all points where $r\sin\theta=0$ due to the $\frac{1}{r\sin\theta}$ that appear in the Jacobian transformation. 

This divergence might pose no problem on cell-centered grids that avoid $r \sin\theta=0$, except that the BSSN equations require that *first and second derivatives* of these quantities be taken. Usual strategies for numerical approximation of these derivatives (e.g., finite difference methods) will "see" these divergences and errors generally will not drop to zero with increased numerical sampling of the functions at points near where the functions diverge.

However, notice that if we define $\lambda^{\phi}$ such that

$$\bar{\Lambda}^{\phi} = \frac{1}{r\sin\theta} \lambda^{\phi},$$

then $\lambda^{\phi}$ will be smooth as well. 

Avoiding such singularities can be generalized, so long as $\lambda^{\phi}$ is defined as:

$$\bar{\Lambda}^{i} = \frac{\lambda^i}{\text{scalefactor[i]}} ,$$

where scalefactor\[i\] is the $i$th scale factor in the given coordinate system. In an identical fashion, we define the smooth versions of $\beta^i$ and $B^i$ to be $\mathcal{V}^i$ and $\mathcal{B}^i$, respectively. We refer to $\mathcal{V}^i$ and $\mathcal{B}^i$ as vet\[i\] and bet\[i\] respectively in the code after the Hebrew letters that bear some resemblance. 

Similarly, we define the smooth versions of $\bar{A}_{ij}$ and $\bar{\varepsilon}_{ij}$ ($a_{ij}$ and $h_{ij}$, respectively) via

\begin{align}
\bar{A}_{ij} &= \text{scalefactor[i]}\ \text{scalefactor[j]}\  a_{ij} \\
\bar{\varepsilon}_{ij} &= \text{scalefactor[i]}\ \text{scalefactor[j]}\  h_{ij},
\end{align}

where in this case we *multiply* due to the fact that these tensors are purely covariant (as opposed to contravariant).

To slightly simplify the notation, in NRPy+ we define *rescaling variable* ReU\[i\] and ReDD\[i\]\[j\], such that

\begin{align}
\text{ReU[i]} &= 1 / \text{scalefactor[i]} \\
\text{ReDD[i][j]} &= \text{scalefactor[i] scalefactor[j]}.
\end{align}

Thus, for example,
\begin{align}
\bar{A}_{ij} &= \text{ReDD[i][j]} a_{ij} \\
\bar{\Lambda}^{i} &= \text{ReU[i]} \lambda^i,
\end{align}
where no sums are implied by the repeated indices.

Further, since the scale factors are *time independent*, 

$$\partial_t \bar{A}_{ij} = \text{ReDD[i][j]}\  \partial_t a_{ij}.$$

Thus in the below implementation of the BSSN equations, we will first define the right-hand sides of the equations for all "evolved" quantities 

$$\left\{\bar{\gamma}_{i j},\bar{A}_{i j},\phi, K, \bar{\Lambda}^{i}, \alpha, \beta^i, B^i\right\},$$ 

and then divide or multiply the indexed evolved quantities by the scale factors according to this prescription so that the evolved variables *in our numerical scheme* are

$$\left\{h_{i j},a_{i j},\phi, K, \lambda^{i}, \alpha, \mathcal{V}^i, \mathcal{B}^i\right\},$$ 
respectively.

Additionally, to ensure that *spatial* derivatives do not sample over a coordinate singularity (where there may be, e.g., a divergence), we simply apply the product/quotient rule. For example,

\begin{align}
\bar{\Lambda}^{i}_{\, ,\, j} &= -\frac{\lambda^i}{(\text{ReU[i]})^2}\ \partial_j \left(\text{ReU[i]}\right) + \frac{\partial_j \lambda^i}{\text{ReU[i]}} \\
&= -\frac{\lambda^i}{(\text{ReU[i]})^2}\ \text{ReU_dD[i][j]} + \frac{\partial_j \lambda^i}{\text{ReU[i]}}
\end{align}

where the derivative $\text{ReU_dD[i][j]}$ **is computed symbolically and exactly** using SymPy, and the derivative $\partial_j \lambda^i$ represents a derivative of a *smooth* quantity (so long as $\bar{\Lambda}^{i}$ is smooth in a Cartesian basis).

### Numerical Implementation: The Lie derivatives

Next we will discuss the numerical implementation of the above equations. First, you'll notice on the left-hand side of the equations includes the time derivatives. Numerically, these equations are solved using as an [initial value problem](https://en.wikipedia.org/wiki/Initial_value_problem), where data are specified at a given time, and then pushed forward in time using the [Method of Lines (MoL)](https://en.wikipedia.org/wiki/Method_of_lines). MoL requires that the equations be written in the form:

$$\partial_t \vec{U} = \vec{f}\left(\vec{U},\partial_i \vec{U}, \partial_i \partial_j \vec{U},...\right),$$

for the vector of "evolved quantities" $\vec{U}$, where the right-hand side vector $\vec{f}$ *does not* contain *explicit* time derivatives of $\vec{U}$.

Thus we must first rewrite the above equations so that partial derivatives of time appear on the left-hand sides, meaning that the Lie derivative terms must be moved to the right-hand sides of the equations.

In this section, we provide explicit expressions for the [Lie derivatives](https://en.wikipedia.org/wiki/Lie_derivative) $\mathcal{L}_\beta$ appearing inside the $\partial_\perp = \partial_t - \mathcal{L}_\beta$ operators for $\left\{\varepsilon_{i j},\bar{A}_{i j},W, K, \bar{\Lambda}^{i}\right\}$.

In short, the Lie derivative of tensor weight $w$ is given by (from [the wikipedia article on Lie derivatives](https://en.wikipedia.org/wiki/Lie_derivative))
\begin{align}
(\mathcal {L}_X T) ^{a_1 \ldots a_r}{}_{b_1 \ldots b_s} &= X^c(\partial_c T^{a_1 \ldots a_r}{}_{b_1 \ldots b_s}) \\
&\quad - (\partial_c X ^{a_1}) T ^{c a_2 \ldots a_r}{}_{b_1 \ldots b_s} - \ldots - (\partial_c X^{a_r}) T ^{a_1 \ldots a_{r-1}c}{}_{b_1 \ldots b_s} \\
 &\quad  +  (\partial_{b_1} X^c) T ^{a_1 \ldots a_r}{}_{c b_2 \ldots b_s} + \ldots + (\partial_{b_s} X^c) T ^{a_1 \ldots a_r}{}_{b_1 \ldots b_{s-1} c} + w (\partial_{c} X^c) T ^{a_1 \ldots a_r}{}_{b_1 \ldots b_{s}}
\end{align}

Thus to evaluate the Lie derivative, one must first know the tensor density weight $w$ for each tensor. In this formulation of Einstein's equations, **all evolved quantities have density weight $w=0$**, so according to the definition of Lie derivative above,
\begin{align}
\mathcal{L}_\beta \bar{\gamma}_{ij} &= \beta^k \partial_k \bar{\gamma}_{ij} + \partial_i \beta^k \bar{\gamma}_{kj} + \partial_j \beta^k \bar{\gamma}_{ik}, \\
\mathcal{L}_\beta \bar{A}_{ij} &= \beta^k \partial_k \bar{A}_{ij} + \partial_i \beta^k \bar{A}_{kj} + \partial_j \beta^k \bar{A}_{ik}, \\
\mathcal{L}_\beta \phi &= \beta^k \partial_k \phi, \\
\mathcal{L}_\beta K &= \beta^k \partial_k K, \\
\mathcal{L}_\beta \bar{\Lambda}^i &= \beta^k \partial_k \bar{\Lambda}^i - \partial_k \beta^i \bar{\Lambda}^k
\end{align}

With these definitions, the BSSN equations for the un-rescaled evolved variables in the form $\partial_t \vec{U} = f\left(\vec{U},\partial_i \vec{U}, \partial_i \partial_j \vec{U},...\right)$ become

\begin{align}
  \partial_t \bar{\gamma}_{i j} {} = {} & \left[\beta^k \partial_k \bar{\gamma}_{ij} + \partial_i \beta^k \bar{\gamma}_{kj} + \partial_j \beta^k \bar{\gamma}_{ik} \right] + \frac{2}{3} \bar{\gamma}_{i j} \left (\alpha \bar{A}_{k}^{k} - \bar{D}_{k} \beta^{k}\right ) - 2 \alpha \bar{A}_{i j} \; , \\
  \partial_t \bar{A}_{i j} {} = {} & \left[\beta^k \partial_k \bar{A}_{ij} + \partial_i \beta^k \bar{A}_{kj} + \partial_j \beta^k \bar{A}_{ik} \right] - \frac{2}{3} \bar{A}_{i j} \bar{D}_{k} \beta^{k} - 2 \alpha \bar{A}_{i k} {\bar{A}^{k}}_{j} + \alpha \bar{A}_{i j} K \nonumber \\
  & + e^{-4 \phi} \left \{-2 \alpha \bar{D}_{i} \bar{D}_{j} \phi + 4 \alpha \bar{D}_{i} \phi \bar{D}_{j} \phi  + 4 \bar{D}_{(i} \alpha \bar{D}_{j)} \phi - \bar{D}_{i} \bar{D}_{j} \alpha + \alpha \bar{R}_{i j} \right \}^{\text{TF}} \; , \\
  \partial_t \phi {} = {} & \left[\beta^k \partial_k \phi \right] + \frac{1}{6} \left (\bar{D}_{k} \beta^{k} - \alpha K \right ) \; , \\
  \partial_{t} K {} = {} & \left[\beta^k \partial_k K \right] + \frac{1}{3} \alpha K^{2} + \alpha \bar{A}_{i j} \bar{A}^{i j} - e^{-4 \phi} \left (\bar{D}_{i} \bar{D}^{i} \alpha + 2 \bar{D}^{i} \alpha \bar{D}_{i} \phi \right ) \; , \\
  \partial_t \bar{\Lambda}^{i} {} = {} & \left[\beta^k \partial_k \bar{\Lambda}^i - \partial_k \beta^i \bar{\Lambda}^k \right] + \bar{\gamma}^{j k} \hat{D}_{j} \hat{D}_{k} \beta^{i} + \frac{2}{3} \Delta^{i} \bar{D}_{j} \beta^{j} + \frac{1}{3} \bar{D}^{i} \bar{D}_{j} \beta^{j} \nonumber \\
  & - 2 \bar{A}^{i j} \left (\partial_{j} \alpha - 6 \partial_{j} \phi \right ) + 2 \bar{A}^{j k} \Delta_{j k}^{i}  -\frac{4}{3} \alpha \bar{\gamma}^{i j} \partial_{j} K \\
\partial_t \alpha &= \left[\beta^i \partial_i \alpha\right] - 2 \alpha K \\
  \partial_{t} \beta^{i} &= \left[\beta^j \partial_j \beta^i\right] + B^{i} \\
  \partial_{t} B^{i} &= \left[\beta^j \partial_j B^i\right] + \frac{3}{4} \partial_{0} \bar{\Lambda}^{i} - \eta B^{i}, \; .
\end{align}

where the terms moved from the left-hand side are enclosed in square braces. Note that $\partial_{0} \bar{\Lambda}^{i}$ in the right-hand side of the $\partial_{t} B^{i}$ equation is computed by adding $\beta^j \partial_j \bar{\Lambda}^i$ to the right-hand side expression given for $\partial_t \bar{\Lambda}^i$, so no explicit time dependence occurs in the right-hand sides of the BSSN evolution equations and the Method of Lines can be applied directly.

Of course the above BSSN evolution equations cannot appear as written above in our actual code, because (as discussed in the previous section on tensor rescaling) tensorial expressions can diverge at coordinate singularities. So the equations above will need to be modified to follow the tensor rescaling prescription as described in the previous section.

## Numerical Implementation: Putting it all together.

Recall that according to our tensor rescaling prescription, each time an unrescaled variable appears within a derivative, it is replaced by a rescaled variable multiplied by the rescaling factor ReU\[\] or ReDD\[\]\[\]. Then the derivative expression is replaced using the product rule, where derivatives of ReU or ReDD are computed exactly and the rescaled variable will be computed using numerical approximations (e.g., finite difference derivatives).

Let's start by importing all the needed modules from NRPy+:

In [1]:
# Step 1: import all needed modules from NRPy+:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
import reference_metric as rfm
rfm.reference_metric()
from outputC import *

Next we'll need to initialize parameters and register the set of evolved gridfunctions, which are all rescaled variables $\left\{h_{i j},a_{i j},\phi, K, \lambda^{i}, \alpha, \mathcal{V}^i, \mathcal{B}^i\right\}$:

In [2]:
# Step 2a: Initialize BSSN_RHS parameters
thismodule = __name__
par.initialize_param(par.glb_param("char", thismodule, "ConformalFactor", "W"))

# Step 2b: Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Step 3: Register all needed *evolved* gridfunctions.
# Step 3a: Register indexed quantities, using ixp.register_... functions
hDD = ixp.register_gridfunctions_for_single_rank2("EVOL","hDD", "sym12")
aDD = ixp.register_gridfunctions_for_single_rank2("EVOL","aDD", "sym12")
lambdaU = ixp.register_gridfunctions_for_single_rank1("EVOL","lambdaU")
vetU = ixp.register_gridfunctions_for_single_rank1("EVOL","vetU")
betU = ixp.register_gridfunctions_for_single_rank1("EVOL","betU")
# Step 3b: Register scalar quantities, using gri.register_gridfunctions()
trK, cf, alpha = gri.register_gridfunctions("EVOL",["trK","cf","alpha"])

# Step 4: Register all *auxiliary* gridfunctions.
# Step 4a: Register indexed quantities, using ixp.register_... functions
#RbarDD = ixp.register_gridfunctions_for_single_rank2("EVOL","RbarDD", "sym12")
# Step 4b: Register scalar quantities, using gri.register_gridfunctions()
detgammabar = gri.register_gridfunctions("AUX",["detgammabar"])

## The conformal Ricci tensor

Next, let's compute perhaps the most complicated expression in the BSSN evolution equations: the conformal Ricci tensor:

\begin{align}
  \bar{R}_{i j} {} = {} & - \frac{1}{2} \bar{\gamma}^{k l} \hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j} + \bar{\gamma}_{k(i} \hat{D}_{j)} \bar{\Lambda}^{k} + \Delta^{k} \Delta_{(i j) k} \nonumber \\
  & + \bar{\gamma}^{k l} \left (2 \Delta_{k(i}^{m} \Delta_{j) m l} + \Delta_{i k}^{m} \Delta_{m j l} \right ) \; .
\end{align}

Let's tackle the first term in this expression first, in particular the $\hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j}$ term.

### The conformal Ricci tensor, part 1: computing the  $\hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j}$ term

This is perhaps the most difficult single term in the BSSN equations to compute, but is simplified by first noting that the covariant derivative of a metric with respect to itself is zero
$$\hat{D}_{l} \hat{\gamma}_{ij} = 0,$$
so 
$$\hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j} = \hat{D}_{k} \hat{D}_{l} \left(\hat{\gamma}_{i j} + \varepsilon_{ij}\right) = \hat{D}_{k} \hat{D}_{l} \varepsilon_{ij}.$$

Next, the covariant derivative of a tensor is given by (from the [wikipedia article on covariant differentiation](https://en.wikipedia.org/wiki/Covariant_derivative)):
\begin{align}
  {(\nabla_{e_c} T)^{a_1 \ldots a_r}}_{b_1 \ldots b_s} = {}
    &\frac{\partial}{\partial x^c}{T^{a_1 \ldots a_r}}_{b_1 \ldots b_s} \\
    &+ \,{\Gamma ^{a_1}}_{dc} {T^{d a_2 \ldots a_r}}_{b_1 \ldots b_s} + \cdots + {\Gamma^{a_r}}_{dc} {T^{a_1 \ldots a_{r-1}d}}_{b_1 \ldots b_s} \\
    &-\,{\Gamma^d}_{b_1 c} {T^{a_1 \ldots a_r}}_{d b_2 \ldots b_s} - \cdots - {\Gamma^d}_{b_s c} {T^{a_1 \ldots a_r}}_{b_1 \ldots b_{s-1} d}.
\end{align}

Therefore, 
$$\hat{D}_{l} \bar{\gamma}_{i j} = \hat{D}_{l} \varepsilon_{i j} = \varepsilon_{i j,l} - \hat{\Gamma}^m_{i l} \varepsilon_{m j} -\hat{\Gamma}^m_{j l} \varepsilon_{i m}.$$

Since the covariant first derivative is a tensor, the covariant second derivative is given by (same as [Eq. 27 in Baumgarte et al (2012)](https://arxiv.org/pdf/1211.6632.pdf))
\begin{align}
\hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j} &= \hat{D}_{k} \hat{D}_{l} \varepsilon_{i j}  \\
&= \partial_k \hat{D}_{l} \varepsilon_{i j}
 - \hat{\Gamma}^m_{lk} \left(\hat{D}_{m} \varepsilon_{i j}\right) 
 - \hat{\Gamma}^m_{ik} \left(\hat{D}_{l} \varepsilon_{m j}\right)
 - \hat{\Gamma}^m_{jk} \left(\hat{D}_{l} \varepsilon_{i m}\right),
\end{align}

where the first term is the partial derivative of the expression already derived for $\hat{D}_{l} \varepsilon_{i j}$:

\begin{align}
\partial_k \hat{D}_{l} \varepsilon_{i j} &= \partial_k \left(\varepsilon_{ij,l} - \hat{\Gamma}^m_{i l} \varepsilon_{m j} -\hat{\Gamma}^m_{j l} \varepsilon_{i m} \right) \\
&= \varepsilon_{ij,lk} - \hat{\Gamma}^m_{i l,k} \varepsilon_{m j} - \hat{\Gamma}^m_{i l} \varepsilon_{m j,k} - \hat{\Gamma}^m_{j l,k} \varepsilon_{i m} - \hat{\Gamma}^m_{j l} \varepsilon_{i m,k}.
\end{align}

In terms of the evolved quantity $h_{ij}$, the derivatives of $\varepsilon_{ij}$ are given by:
\begin{align}
\varepsilon_{ij,k} &= \partial_k \left(h_{ij} \text{ReDD[i][j]}\right) \\
&= h_{ij,k} \text{ReDD[i][j]} + h_{ij} \text{ReDD_dD[i][j][k]},
\end{align}
and
\begin{align}
\varepsilon_{ij,kl} &= \partial_l \left(h_{ij,k} \text{ReDD[i][j]} + h_{ij} \text{ReDD_dD[i][j][k]} \right)\\
&= h_{ij,kl} \text{ReDD[i][j]} + h_{ij,k} \text{ReDD_dD[i][j][l]} + h_{ij,l} \text{ReDD_dD[i][j][k]} + h_{ij} \text{ReDD_dDD[i][j][k][l]}.
\end{align}

Before returning to NRPy+, let's first define some NRPy+ notation:
* $\hat{D}_{l} \bar{\gamma}_{i j} = \bar{\gamma}_{ij;\hat{l}} = \text{gammabarDD_DhatD[i][j][l]}$
* $\partial_k \hat{D}_{l} \bar{\gamma}_{i j} = \bar{\gamma}_{ij;\hat{l},k} = \text{gammabarDD_dHatD_dD[i][j][l][k]}$
* $\hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j} = \bar{\gamma}_{ij;\hat{l}\hat{k}} = \text{gammabarDD_dHatDD[i][j][l][k]}$

In [3]:
RbarDD = ixp.zerorank2()

# Step 5a: Define \varepsilon_{ij} = epsDD[i][j]
epsDD = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        epsDD[i][j] = hDD[i][j]*rfm.ReDD[i][j]

# Step 5b: Define epsDD_dD[i][j][k]
hDD_dD = ixp.declarerank3("hDD_dD","sym12")
epsDD_dD = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            epsDD_dD[i][j][k] = hDD_dD[i][j][k]*rfm.ReDD[i][j] + hDD[i][j]*rfm.ReDDdD[i][j][k]

# Step 5c: Define epsDD_dDD[i][j][k][l]
hDD_dDD = ixp.declarerank4("hDD_dDD","sym12_sym34")
epsDD_dDD = ixp.zerorank4()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                epsDD_dDD[i][j][k][l] = hDD_dDD[i][j][k][l]*rfm.ReDD[i][j] + \
                                        hDD_dD[i][j][k]*rfm.ReDDdD[i][j][l] + \
                                        hDD_dD[i][j][l]*rfm.ReDDdD[i][j][k] + \
                                        hDD[i][j]*rfm.ReDDdDD[i][j][k][l]

# Step 5d: Define DhatgammabarDDdD[i][j][l] = \bar{\gamma}_{ij;\hat{l}}
gammabarDD_dHatD = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for l in range(DIM):
            gammabarDD_dHatD[i][j][l] = epsDD_dD[i][j][l]
            for m in range(DIM):
                gammabarDD_dHatD[i][j][l] -= rfm.GammahatUDD[m][i][l]*epsDD[m][j] \
                                          -  rfm.GammahatUDD[m][j][l]*epsDD[i][m]

# Step 5e: Define \bar{\gamma}_{ij;\hat{l},k} = DhatgammabarDD_dHatD_dD[i][j][l][k]
gammabarDD_dHatD_dD = ixp.zerorank4()
for i in range(DIM):
    for j in range(DIM):
        for l in range(DIM):
            for k in range(DIM):
                gammabarDD_dHatD_dD[i][j][l][k] = epsDD_dDD[i][j][l][k]
                for m in range(DIM):
                    gammabarDD_dHatD_dD[i][j][l][k] += -rfm.GammahatUDDdD[m][i][l][k]*epsDD[m][j]  \
                                                       -rfm.GammahatUDD[m][i][l]*epsDD_dD[m][j][k] \
                                                       -rfm.GammahatUDDdD[m][j][l][k]*epsDD[i][m]  \
                                                       -rfm.GammahatUDD[m][j][l]*epsDD_dD[i][m][k]
# Step 5f: Define \bar{\gamma}_{ij;\hat{l}\hat{k}} = DhatgammabarDD_dHatDD[i][j][l][k]
gammabarDD_dHatDD = ixp.zerorank4()
for i in range(DIM):
    for j in range(DIM):
        for l in range(DIM):
            for k in range(DIM):
                gammabarDD_dHatDD[i][j][l][k] = gammabarDD_dHatD_dD[i][j][l][k]
                for m in range(DIM):
                    gammabarDD_dHatDD[i][j][l][k] += - rfm.GammahatUDD[m][l][k]*gammabarDD_dHatD[i][j][m] \
                                                     - rfm.GammahatUDD[m][i][k]*gammabarDD_dHatD[m][j][l] \
                                                     - rfm.GammahatUDD[m][j][k]*gammabarDD_dHatD[i][m][l]

# Step 5g: Compute \bar{\gamma}_{ij} and its inverse (using built-in function ixp.symm_matrix_inverter3x3()):
gammabarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        gammabarDD[i][j] = hDD[i][j]*rfm.ReDD[i][j] + rfm.ghatDD[i][j]
gammabarUU, dummydet = ixp.symm_matrix_inverter3x3(gammabarDD)

# Step 5h: Add the first term to RbarDD:
#         - \frac{1}{2} \bar{\gamma}^{k l} \hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j}
RbarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                RbarDD[i][j] += -sp.Rational(1,2) * gammabarUU[k][l]*gammabarDD_dHatDD[i][j][l][k]

### The conformal Ricci tensor, part 2: computing the  $\bar{\gamma}_{k(i} \hat{D}_{j)} \bar{\Lambda}^{k}$ term

By definition, the index symmetrization operation is given by:
$$\bar{\gamma}_{k(i} \hat{D}_{j)} \bar{\Lambda}^{k} = \frac{1}{2} \left( \bar{\gamma}_{ki} \hat{D}_{j} \bar{\Lambda}^{k} + \bar{\gamma}_{kj} \hat{D}_{i} \bar{\Lambda}^{k} \right),$$

and $\bar{\gamma}_{ij}$ is trivially computed ($=\varepsilon_{ij} + \hat{\gamma}_{ij}$) so the challenging part to computing this term is in evaluating $\hat{D}_{j} \bar{\Lambda}^{k}$.

The covariant derivative is with respect to the hatted metric (i.e. the reference metric), so
$$\hat{D}_{j} \bar{\Lambda}^{k} = \partial_j \bar{\Lambda}^{k} + \hat{\Gamma}^{k}_{mj} \bar{\Lambda}^m,$$
except we cannot take derivatives of $\bar{\Lambda}^{k}$ directly due to potential issues with coordinate singularities. Instead we write it in terms of the rescaled quantity $\lambda^k$ via
$$\bar{\Lambda}^{k} = \lambda^k \text{ReU[k]}.$$

Then the expression for $\hat{D}_{j} \bar{\Lambda}^{k}$ becomes
$$
\hat{D}_{j} \bar{\Lambda}^{k} = \lambda^{k}_{,j} \text{ReU[k]} + \lambda^{k} \text{ReUdD[k][j]} + \hat{\Gamma}^{k}_{mj} \lambda^{m} \text{ReU[m]},
$$
and the NRPy+ code for this expression is written

In [4]:
# Step 6a: Second term of RhatDD: compute \hat{D}_{j} \bar{\Lambda}^{k} = LambarU_dHatD[k][j]
LambarU_dHatD = ixp.zerorank2()
lambdaU_dD = ixp.declarerank2("lambdaU_dD","sym12")
for j in range(DIM):
    for k in range(DIM):
        LambarU_dHatD[k][j] = lambdaU_dD[j][k]*rfm.ReU[k] + lambdaU[k]*rfm.ReUdD[k][j]
        for m in range(DIM):
            LambarU_dHatD[k][j] += rfm.GammahatUDD[k][m][j]*lambdaU[m]*rfm.ReU[m]

Next we add the second term, $\frac{1}{2} \left( \bar{\gamma}_{ki} \hat{D}_{j} \bar{\Lambda}^{k} + \bar{\gamma}_{kj} \hat{D}_{i} \bar{\Lambda}^{k} \right)$, to the Ricci tensor:

In [5]:
# Step 6b: Add the second term to the Ricci tensor
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            RbarDD[i][j] += sp.Rational(1,2) * (gammabarDD[k][i]*LambarU_dHatD[k][j] + \
                                                gammabarDD[k][j]*LambarU_dHatD[k][i])

### The conformal Ricci tensor, part 3: computing the  remaining terms: $\Delta^{k} \Delta_{(i j) k}  + \bar{\gamma}^{k l} \left (2 \Delta_{k(i}^{m} \Delta_{j) m l} + \Delta_{i k}^{m} \Delta_{m j l} \right )$

Since $\Delta^k_{ij}$ is defined as a difference in Christoffel symbols ("$\Gamma$"s) In NRPy+, we define them as
* $\Delta^{k}_{ij} = \bar{\Gamma}^i_{jk} - \hat{\Gamma}^i_{jk} = $DGammaUDD\[k\]\[i\]\[j\]
* $\Delta^{k} = \bar{\gamma}^{ij} \Delta^{k}_{ij} = $DGammaU\[k\]

Adding these expressions to Ricci is straightforward: first we define $\bar{\gamma}_{ij,k}$ in terms of the evolved variable $h_{ij}$ its derivative, and reference-metric quantities
\begin{align}
\bar{\gamma}_{ij,k} &= \partial_k \bar{\gamma}_{ij} \\
&= \partial_k \left(h_{ij} \text{ReDD[i][j]} + \hat{\gamma}_{ij}\right) \\
&= h_{ij,k} \text{ReDD[i][j]} + h_{ij} \text{ReDDdD[i][j][k]} + \hat{\gamma}_{ij,k}.
\end{align}
From this we then define $\bar{\Gamma}^i_{jk}$, third $\Delta^k_{ij}$ and $\Delta^k$, fourth $\Delta_{ijk}=\bar{\gamma}_{im}\Delta^m_{jk}$, and finally we add them as prescribed to the conformal Ricci tensor $\bar{R}_{ij}$:

In [6]:
# Step 7a: Define \bar{\gamma}_{ij,k} = gammabarDDdD[i][j][k] 
#          = h_{ij,k} \text{ReDD[i][j]} + h_{ij} \text{ReDDdD[i][j][k]} + \hat{\gamma}_{ij,k}.
gammabarDD_dD = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            gammabarDD_dD[i][j][k] = hDD_dD[i][j][k]*rfm.ReDD[i][j] + hDD[i][j]*rfm.ReDDdD[i][j][k] \
                                   + rfm.ghatDDdD[i][j][k]
# Step 7b: Define barred Christoffel symbol \bar{\Gamma}^{i}_{jk} = GammabarUDD[i][j][k]
GammabarUDD = ixp.zerorank3()
for i in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            for m in range(DIM):
                GammabarUDD[i][k][l] += (sp.Rational(1,2))*gammabarUU[i][m]* \
                                        (gammabarDD_dD[m][k][l] + gammabarDD_dD[m][l][k] - gammabarDD_dD[k][l][m])

# Step 7c: Define \Delta^i_{jk} = \bar{\Gamma}^i_{jk} - \hat{\Gamma}^i_{jk} = DGammaUDD[i][j][k]
DGammaUDD = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            DGammaUDD[i][j][k] = GammabarUDD[i][j][k] - rfm.GammahatUDD[i][j][k]

# Step 7d: Define \Delta^i = \bar{\gamma}^{jk} \Delta^i_{jk}
DGammaU = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            DGammaU[i] += gammabarUU[j][k] * DGammaUDD[i][j][k]

# Step 7e: Define \Delta_{ijk} = \bar{\gamma}_{im} \Delta^m_{jk}
DGammaDDD = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for m in range(DIM):
                DGammaDDD[i][j][k] += gammabarDD[i][m] * DGammaUDD[m][j][k]

# Step 7e: Add third term to Ricci tensor: \Delta^{k} \Delta_{(i j) k} = 1/2 \Delta^{k} (\Delta_{i j k} + \Delta_{j i k})
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            RbarDD[i][j] += sp.Rational(1,2) * DGammaU[k] * (DGammaDDD[i][j][k] + DGammaDDD[j][i][k])

# Step 7f: Add remaining terms to Ricci tensor: 
# \bar{\gamma}^{k l} (\Delta^{m}_{k i} \Delta_{j m l}
#                   + \Delta^{m}_{k j} \Delta_{i m l} 
#                   + \Delta^{m}_{i k} \Delta_{m j l})
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                for m in range(DIM):
                    RbarDD[i][j] += gammabarUU[k][l] * (DGammaUDD[m][k][i]*DGammaDDD[j][m][l] + 
                                                        DGammaUDD[m][k][j]*DGammaDDD[i][m][l] + 
                                                        DGammaUDD[m][i][k]*DGammaDDD[m][j][l])

## Right-hand sides of BSSN equations, part 1: $\partial_t \bar{\gamma}_{i j}$

Now that the most difficult quantity to compute is out of the way, we now have a template for moving forward with the rest of the BSSN equations. In the following, we will evaluate the right-hand sides of each non-evolved variable, and then rescale the result to correspond to the evolved variables as our last step. 

Before we get started, one final numerical detail: all shift advection terms (i.e., terms of the form $\beta^i \partial_i$) are upwinded according to the direction of the shift vector. Upwinded derivatives are given by the "_dupD" variable suffix.

Let's start with

$$
\partial_t \bar{\gamma}_{i j} = 
{\underbrace {\textstyle \left[\beta^k \partial_k \bar{\gamma}_{ij} + \partial_i \beta^k \bar{\gamma}_{kj} + \partial_j \beta^k \bar{\gamma}_{ik} \right]}_{\text{Term 1}}} + 
{\underbrace {\textstyle \frac{2}{3} \bar{\gamma}_{i j} \left (\alpha \bar{A}_{k}^{k} - \bar{D}_{k} \beta^{k}\right )}_{\text{Term 2}}}
{\underbrace {\textstyle -2 \alpha \bar{A}_{i j}}_{\text{Term 3}}}.
$$

### Term 1 of $\partial_t \bar{\gamma}_{i j} =$ gammabar_rhsDD\[i\]\[j\]: $\beta^k \bar{\gamma}_{ij,k} + \beta^k_{,i} \bar{\gamma}_{kj} + \beta^k_{,j} \bar{\gamma}_{ik}$

We first need to define the rescaled version of $\beta^i$, $\mathcal{B}^i$=vetU\[i\] and betaU_dD\[i\]\[j\] in terms of the rescaled derivative.

In [7]:
# Step 8a: Define \beta^i and \beta^i_{,k} in terms of rescaled quantity vetU[i] and vetU_dD[i][j]:
betaU = ixp.zerorank1()
for i in range(DIM):
    betaU[i] = vetU[i]*rfm.ReU[i]

vetU_dD = ixp.declarerank2("vetU_dD","none")
betaU_dD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        betaU_dD[i][j] = vetU_dD[i][j]*rfm.ReU[i] + vetU[i]*rfm.ReUdD[i][j]
    
# Step 8b: First term of \partial_t \bar{\gamma}_{i j} right-hand side:
# \beta^k \bar{\gamma}_{ij,k} + \beta^k_{,i} \bar{\gamma}_{kj} + \beta^k_{,j} \bar{\gamma}_{ik}
gammabar_rhsDD = ixp.zerorank2()
gammabarDD_dupD = ixp.declarerank3("gammabarDD_dupD","sym12")
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            gammabar_rhsDD[i][j] += betaU[k]*gammabarDD_dupD[i][j][k] + betaU_dD[k][i]*gammabarDD[k][j] \
                                                                      + betaU_dD[k][j]*gammabarDD[i][k]

### Term 2 of $\partial_t \bar{\gamma}_{i j} =$ gammabar_rhsDD\[i\]\[j\]: $\frac{2}{3} \bar{\gamma}_{i j} \left (\alpha \bar{A}_{k}^{k} - \bar{D}_{k} \beta^{k}\right )$

Let's first convert this expression to be in terms of the evolved variables $a_{ij}$ and $\mathcal{B}^i$, starting with $\bar{A}_{ij} = a_{ij} \text{ReDD[i][j]}$. Then $\bar{A}^k_{k} = \bar{\gamma}^{ij} \bar{A}_{ij}$, and we have already defined $\bar{\gamma}^{ij}$ in terms of the evolved quantity $h_{ij}$.

Next, we wish to compute 

$$\bar{D}_{k} \beta^{k} = \beta^k_{,k} + \frac{\beta^k \bar{\gamma}_{,k}}{2 \bar{\gamma}},$$

where $\bar{\gamma}$ is the determinant of the conformal metric $\bar{\gamma}_{ij}$, which is assumed computed outside this function and set as the gridfunction detgammabar. *Exercise to student: Prove the above relation.*

In [8]:
# Step 8c: Define \bar{A}_{ij} = a_{ij} \text{ReDD[i][j]} = AbarDD[i][j], and its contraction trAbar = \bar{A}^k_k
AbarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        AbarDD[i][j] = aDD[i][j]*rfm.ReDD[i][j]
trAbar = sp.sympify(0)
for i in range(DIM):
    for j in range(DIM):
        trAbar += gammabarUU[i][j]*AbarDD[i][j]

# Step 8d: Compute the contraction \bar{D}_k \beta^k = \beta^k_{,k} + \frac{\beta^k \bar{\gamma}_{,k}}{2 \bar{\gamma}}
detgammabar_dD = ixp.declarerank1("detgammabar_dD")
Dbarbetacontraction = sp.sympify(0)
for k in range(DIM):
    Dbarbetacontraction += betaU_dD[k][k] + betaU[k]*detgammabar_dD[k]/(2*detgammabar)

# Step 8d: Second term of \partial_t \bar{\gamma}_{i j} right-hand side:
# \frac{2}{3} \bar{\gamma}_{i j} \left (\alpha \bar{A}_{k}^{k} - \bar{D}_{k} \beta^{k}\right )
for i in range(DIM):
    for j in range(DIM):
        gammabar_rhsDD[i][j] += sp.Rational(2,3)*gammabarDD[i][j]*(alpha*trAbar - Dbarbetacontraction)

### Term 3 of $\partial_t \bar{\gamma}_{i j} =$ gammabar_rhsDD\[i\]\[j\]: $-2 \alpha \bar{A}_{ij}$

In [9]:
# Step 8e: Third term of \partial_t \bar{\gamma}_{i j} right-hand side:
# -2 \alpha \bar{A}_{ij}
for i in range(DIM):
    for j in range(DIM):
        gammabar_rhsDD[i][j] += -2*alpha*AbarDD[i][j]

## Right-hand sides of BSSN equations, part 2: $\partial_t \bar{A}_{i j}$

$$\partial_t \bar{A}_{i j} = 
{\underbrace {\textstyle \left[\beta^k \partial_k \bar{A}_{ij} + \partial_i \beta^k \bar{A}_{kj} + \partial_j \beta^k \bar{A}_{ik} \right]}_{\text{Term 1}}}
{\underbrace {\textstyle - \frac{2}{3} \bar{A}_{i j} \bar{D}_{k} \beta^{k} - 2 \alpha \bar{A}_{i k} {\bar{A}^{k}}_{j} + \alpha \bar{A}_{i j} K}_{\text{Term 2}}} + 
{\underbrace {\textstyle e^{-4 \phi} \left \{-2 \alpha \bar{D}_{i} \bar{D}_{j} \phi + 4 \alpha \bar{D}_{i} \phi \bar{D}_{j} \phi  + 4 \bar{D}_{(i} \alpha \bar{D}_{j)} \phi - \bar{D}_{i} \bar{D}_{j} \alpha + \alpha \bar{R}_{i j} \right \}^{\text{TF}}}_{\text{Term 3}}}$$

### Term 1 of $\partial_t \bar{A}_{i j}$ = Abar_rhsDD\[i\]\[j\]: $\left[\beta^k \partial_k \bar{A}_{ij} + \partial_i \beta^k \bar{A}_{kj} + \partial_j \beta^k \bar{A}_{ik} \right]$

Notice the first subexpression has a $\beta^k \partial_k A_{ij}$ advection term, which will be upwinded.

In [10]:
# Step 9a: First term of \partial_t \bar{A}_{i j}:
# \beta^k \partial_k \bar{A}_{ij} + \partial_i \beta^k \bar{A}_{kj} + \partial_j \beta^k \bar{A}_{ik}
Abar_rhsDD = ixp.zerorank2()
AbarDD_dupD = ixp.declarerank3("AbarDD_dupD","sym12")
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            Abar_rhsDD[i][j] += betaU[k]*AbarDD_dupD[i][j][k] + betaU_dD[k][i]*AbarDD[k][j] \
                                                              + betaU_dD[k][j]*AbarDD[i][k]

### Term 2 of $\partial_t \bar{A}_{i j}$ = Abar_rhsDD\[i\]\[j\]: $- \frac{2}{3} \bar{A}_{i j} \bar{D}_{k} \beta^{k} - 2 \alpha \bar{A}_{i k} \bar{A}^{k}_{j} + \alpha \bar{A}_{i j} K$

Note that $\bar{D}_{k} \beta^{k}$ was already defined as "Dbarbetacontraction".

In [11]:
# Step 9b: Second term of \partial_t \bar{A}_{i j}:
# - (2/3) \bar{A}_{i j} \bar{D}_{k} \beta^{k} - 2 \alpha \bar{A}_{i k} {\bar{A}^{k}}_{j} + \alpha \bar{A}_{i j} K
AbarUD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            AbarUD[i][j] += gammabarUU[i][k]*AbarDD[k][j]

for i in range(DIM):
    for j in range(DIM):
        Abar_rhsDD[i][j] += -sp.Rational(2,3)*AbarDD[i][j]*Dbarbetacontraction + alpha*AbarDD[i][j]*trK
        for k in range(DIM):
            Abar_rhsDD[i][j] += -2*alpha * AbarDD[i][k]*AbarUD[k][j]

### Term 3 of $\partial_t \bar{A}_{i j}$ = Abar_rhsDD\[i\]\[j\]: $e^{-4 \phi} \left \{-2 \alpha \bar{D}_{i} \bar{D}_{j} \phi + 4 \alpha \bar{D}_{i} \phi \bar{D}_{j} \phi  + 4 \bar{D}_{(i} \alpha \bar{D}_{j)} \phi - \bar{D}_{i} \bar{D}_{j} \alpha + \alpha \bar{R}_{i j} \right \}^{\text{TF}}$

The first covariant derivatives of $\phi$ and $\alpha$ are simply partial derivatives. However, $\phi$ is not a gridfunction; "cf" is. cf="W" (default value) denotes that the evolved variable is $W=e^{-2 \phi}$, which results in smoother spacetime fields around puncture black holes (desirable).

In [ ]:
# Step 9c: Define partial derivatives of \phi in terms of evolved quantity "cf":
cf_dD = ixp.declarerank1("cf_dD")
cf_dupD = ixp.declarerank1("cf_dupD") # Needed for \partial_t \phi next.
cf_dDD = ixp.declarerank2("cf_dDD","sym12")
phi_dD = ixp.zerorank1()
phi_dupD = ixp.zerorank1()
phi_dDD = ixp.zerorank2()
exp_m4phi = sp.sympify(0)
if par.parval_from_str("ConformalFactor") == "phi":
    for i in range(DIM):
        phi_dD[i] = cf_dD[i]
        phi_dupD[i] = cf_dupD[i]
        for j in range(DIM):
            phi_dDD[i][j] = cf_dDD[i][j]
    exp_m4phi = sp.exp(-4*cf)
elif par.parval_from_str("ConformalFactor") == "W":
    # \partial_i W = \partial_i (e^{-2 phi}) = -2 e^{-2 phi} \partial_i phi
    # -> \partial_i phi = -\partial_i cf / (2 cf)
    for i in range(DIM):
        phi_dD[i] = - cf_dD[i] / (2*cf)
        phi_dupD[i] = - cf_dupD[i] / (2*cf)
        for j in range(DIM):
            # \partial_j \partial_i phi = - \partial_j [\partial_i cf / (2 cf)]
            #                           = - cf_{,ij} / (2 cf) + \partial_i cf \partial_j cf / (2 cf^2)
            phi_dDD[i][j] = (- cf_dDD[i][j] + cf_dD[i]*cf_dD[j] / cf) / (2*cf)
    exp_m4phi = cf*cf
elif par.parval_from_str("ConformalFactor") == "chi":
    # \partial_i chi = \partial_i (e^{-4 phi}) = -4 e^{-4 phi} \partial_i phi
    # -> \partial_i phi = -\partial_i cf / (4 cf)
    for i in range(DIM):
        phi_dD[i] = - cf_dD[i] / (4*cf)
        phi_dupD[i] = - cf_dupD[i] / (4*cf)
        for j in range(DIM):
            # \partial_j \partial_i phi = - \partial_j [\partial_i cf / (4 cf)]
            #                           = - cf_{,ij} / (4 cf) + \partial_i cf \partial_j cf / (4 cf^2)
            phi_dDD[i][j] = (- cf_dDD[i][j] + cf_dD[i]*cf_dD[j] / cf) / (4*cf)
    exp_m4phi = cf
else:
    print("Error: ConformalFactor == "+par.parval_from_str("ConformalFactor")+" unsupported!")
    exit(1)
    
# Step 9d: Define phi_dBarD = phi_dD (since phi is a scalar) and phi_dBarDD (covariant derivative)
#          \bar{D}_i \bar{D}_j \phi = \phi_{;\bar{i}\bar{j}} = \bar{D}_i \phi_{,j}
#                                   = \phi_{,ij} - \bar{\Gamma}^k_{ij} \phi_{,k}
phi_dBarD = phi_dD
phi_dBarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        phi_dBarDD[i][j] = phi_dDD[i][j]
        for k in range(DIM):
            phi_dBarDD[i][j] += - GammabarUDD[k][i][j]*phi_dD[k]

# Step 9e: Define first and second derivatives of \alpha, as well as 
#         \bar{D}_i \bar{D}_j \alpha, which is defined just like phi
alpha_dD = ixp.declarerank1("alpha_dD")
alpha_dDD = ixp.declarerank2("alpha_dDD","sym12")
alpha_dBarD = alpha_dD
alpha_dBarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        alpha_dBarDD[i][j] = alpha_dDD[i][j]
        for k in range(DIM):
            alpha_dBarDD[i][j] += - GammabarUDD[k][i][j]*alpha_dD[k]

# Step 9f: Define the terms in curly braces:
curlybrackettermsDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        curlybrackettermsDD[i][j] = -2*alpha*phi_dBarDD[i][j] + 4*alpha*phi_dBarD[i]*phi_dBarD[j] \
                                    +2*alpha_dBarD[i]*phi_dBarD[j] \
                                    +2*alpha_dBarD[j]*phi_dBarD[i] \
                                    -alpha_dBarDD[i][j] + alpha*RbarDD[i][j]

# Step 9g: Compute the trace:
curlybracketterms_trace = sp.sympify(0)
for i in range(DIM):
    for j in range(DIM):
        curlybracketterms_trace += gammabarUU[i][j]*curlybrackettermsDD[i][j]

# Step 9h: Third and final term of Abar_rhsDD[i][j]:
for i in range(DIM):
    for j in range(DIM):
        Abar_rhsDD[i][j] += exp_m4phi*(curlybrackettermsDD[i][j] - \
                                       sp.Rational(1,3)*gammabarDD[i][j]*curlybracketterms_trace)   

## Right-hand sides of BSSN equations, part 3: $\partial_t \phi \to \partial_t$ cf

$$\partial_t \phi = 
{\underbrace {\textstyle \left[\beta^k \partial_k \phi \right]}_{\text{Term 1}}} + 
{\underbrace {\textstyle \frac{1}{6} \left (\bar{D}_{k} \beta^{k} - \alpha K \right)}_{\text{Term 2}}}$$

The right-hand side of $\partial_t \phi$ is trivial except for the fact that the actual evolved variable is "cf" (short for conformal factor), which could represent
* cf = $\phi$
* cf = $W = e^{-2 \phi}$ (default)
* cf = $\chi = e^{-4 \phi}$

Thus we are actually computing the right-hand side of the equation $\partial_t $cf, which is related to $\partial_t \phi$ via simple relations:
* cf = $\phi$: $\partial_t $cf$ = \partial_t \phi$ (unchanged)
* cf = $W$: $\partial_t $cf$ = \partial_t (e^{-2 \phi}) = -2 e^{-2\phi}\partial_t \phi = -2 W \partial_t \phi$. Thus we need to multiply the right-hand side by $-2 W = -2$cf when cf = $W$.
* cf = $\chi$: Same argument as for $W$, except the right-hand side must be multiplied by $-4 \chi=-4$cf.

In [ ]:
# Step 10: right-hand side of conformal factor variable "cf". Supported
#          options include: cf=phi, cf=W=e^(-2*phi) (default), and cf=chi=e^(-4*phi)
# \partial_t phi = \left[\beta^k \partial_k \phi \right] 
#                  + \frac{1}{6} \left (\bar{D}_{k} \beta^{k} - \alpha K \right )
cf_rhs = sp.Rational(1,6) * (Dbarbetacontraction + alpha*trK) # Term 2
for k in range(DIM):
    cf_rhs += betaU[k]*phi_dupD[k] # Term 1

# Next multiply to convert phi_rhs to cf_rhs.
if par.parval_from_str("ConformalFactor") == "phi":
    pass # do nothing; cf_rhs = phi_rhs
elif par.parval_from_str("ConformalFactor") == "W":
    cf_rhs *= -2*cf # cf_rhs = -2*cf*phi_rhs
elif par.parval_from_str("ConformalFactor") == "chi":
    cf_rhs *= -4*cf # cf_rhs = -4*cf*phi_rhs
else:
    print("Error: ConformalFactor == "+par.parval_from_str("ConformalFactor")+" unsupported!")
    exit(1) 

## Right-hand sides of BSSN equations, part 4: $\partial_t K$

$$
\partial_{t} K = 
{\underbrace {\textstyle \left[\beta^k \partial_k K \right]}_{\text{Term 1}}} + 
{\underbrace {\textstyle \frac{1}{3} \alpha K^{2}}_{\text{Term 2}}} +
{\underbrace {\textstyle \alpha \bar{A}_{i j} \bar{A}^{i j}}_{\text{Term 3}}}
{\underbrace {\textstyle - e^{-4 \phi} \left (\bar{D}_{i} \bar{D}^{i} \alpha + 2 \bar{D}^{i} \alpha \bar{D}_{i} \phi \right )}_{\text{Term 4}}}
$$

In [ ]:
trK_rhs = sp.Rational(1,3)*alpha*trK*trK # Term 2
trK_dupD = ixp.declarerank1("trK_dupD")
for k in range(DIM):
    trK_rhs += betaU[k]*trK_dupD[k]*trK # Term 1
for i in range(DIM):
    for j in range(DIM):
        trK_rhs += -exp_m4phi*gammabarUU[i][j]*(alpha_dBarDD[i][j] + 2*alpha_dBarD[j]*phi_dBarD[i]) # Term 4
AbarUU = ixp.zerorank2() # Needed also for \partial_t \bar{\Lambda}^i
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                AbarUU[i][j] += gammabarUU[i][k]*gammabarUU[j][l]*AbarDD[k][l]
for i in range(DIM):
    for j in range(DIM):
        trK_rhs += alpha*AbarDD[i][j]*AbarUU[i][j] # Term 3

## Right-hand sides of BSSN equations, part 5: $\partial_t \bar{\Lambda}^{i}$

\begin{align}
\partial_t \bar{\Lambda}^{i} &= \left[\beta^k \partial_k \bar{\Lambda}^i - \partial_k \beta^i \bar{\Lambda}^k \right] + \bar{\gamma}^{j k} \hat{D}_{j} \hat{D}_{k} \beta^{i} + \frac{2}{3} \Delta^{i} \bar{D}_{j} \beta^{j} + \frac{1}{3} \bar{D}^{i} \bar{D}_{j} \beta^{j} \nonumber \\
  & - 2 \bar{A}^{i j} \left (\partial_{j} \alpha - 6 \partial_{j} \phi \right ) + 2 \bar{A}^{j k} \Delta_{j k}^{i}  -\frac{4}{3} \alpha \bar{\gamma}^{i j} \partial_{j} K
\end{align}

## Numerical Implementation: Spatial derivatives of barred quantities

As described above, barred quantities may be expressed in terms of products or quotients of reference metric scale factors multiplied by smooth quantities. Thus derivatives of barred quantities must be expanded in terms of an approximate derivative of a smooth quantity multiplied by an exact expression for the derivative of the scale factor (actually, the rescaling variables ReU\[i\] and ReDD\[i\]\[j\], which are trivially defined in terms of the scale factors, above).

